<a href="https://colab.research.google.com/github/NishatVasker/BCI_EEG_CHANNEL_IMAGE_MAKING/blob/main/EEG_CHANNEL_IMAGE_MAKING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [3]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import zipfile
from PIL import Image

# 📌 Create base output directory
base_output_dir = "output"
os.makedirs(base_output_dir, exist_ok=True)

# 🎨 **Fixed Color Mapping for Each Channel**
channel_colors = {
    "Delta_TP9": "red", "Delta_AF7": "blue", "Delta_AF8": "green", "Delta_TP10": "purple",
    "Theta_TP9": "orange", "Theta_AF7": "cyan", "Theta_AF8": "magenta", "Theta_TP10": "black",
    "Alpha_TP9": "lime", "Alpha_AF7": "brown", "Alpha_AF8": "pink", "Alpha_TP10": "navy",
    "Beta_TP9": "gold", "Beta_AF7": "darkred", "Beta_AF8": "teal", "Beta_TP10": "indigo",
    "Gamma_TP9": "darkblue", "Gamma_AF7": "darkgreen", "Gamma_AF8": "salmon", "Gamma_TP10": "steelblue",
    "RAW_TP9": "olive", "RAW_AF7": "orchid", "RAW_AF8": "coral", "RAW_TP10": "dodgerblue",
    "AUX_RIGHT": "slategray", "Accelerometer_X": "yellow", "Accelerometer_Y": "chocolate", "Accelerometer_Z": "deepskyblue",
    "Gyro_X": "khaki", "Gyro_Y": "fuchsia", "Gyro_Z": "plum", "HeadBandOn": "mediumspringgreen",
    "HSI_TP9": "chartreuse", "HSI_AF7": "mediumvioletred", "HSI_AF8": "mediumblue", "HSI_TP10": "goldenrod",
    "Battery": "lawngreen", "Elements": "firebrick"
}

# ✅ Function to Process Multiple EEG Files
def process_eeg(files, channels, chunk_size):
    if not files:
        return "❌ Error: Please upload at least one CSV file."

    if not channels:
        return "❌ Error: Please select at least one EEG channel."

    output_folders = []

    # 📌 Process Each Uploaded File
    all_image_files = []
    for file in files:
        df = pd.read_csv(file.name)

        # Ensure valid file
        if df.shape[1] < 2:
            continue

        # Extract Time Column
        time_column = df.columns[0]

        # Extract file name without extension for output folder
        file_name = os.path.basename(file.name).replace('.csv', '')

        # Create a separate folder for each file in the output directory
        output_folder = os.path.join(base_output_dir, file_name)
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)  # Clear old data
        os.makedirs(output_folder)

        output_folders.append(output_folder)

        # 📌 Generate EEG Images for Selected Channels
        total_chunks = len(df) // int(chunk_size)
        image_files = []

        for channel in channels:
            if channel not in df.columns:
                continue

            for i in range(total_chunks):
                chunk = df.iloc[i * int(chunk_size) : (i + 1) * int(chunk_size)]

                # Plot EEG Data with Unique Channel Color
                plt.figure(figsize=(10, 5))
                color = channel_colors.get(channel, "black")  # Use mapped color, default to black
                plt.plot(chunk[time_column], chunk[channel], color=color, linestyle='-', linewidth=1)
                plt.axis('off')  # Hide X and Y axes
                plt.xticks([])
                plt.yticks([])

                # Save Image
                img_filename = f"{output_folder}/EEG_{channel}_chunk_{i+1}.png"
                plt.savefig(img_filename, bbox_inches='tight', pad_inches=0)
                plt.close()

                image_files.append(img_filename)

        all_image_files.extend(image_files)

    # ✅ Create a ZIP file containing all processed images
    zip_filename = "eeg_processed_data.zip"
    zip_path = os.path.join(base_output_dir, zip_filename)

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for folder in output_folders:
            for root, _, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=os.path.relpath(file_path, base_output_dir))

    return zip_path, all_image_files  # Returns ZIP file & image list

# ✅ Function to Load EEG Channels for Selection (Multi-Select)
def get_channels(files):
    if not files:
        return gr.update(choices=[], value=[])

    df = pd.read_csv(files[0].name)  # Read first file to get channels
    channels = list(df.columns[1:])  # Return all EEG channels (excluding time column)

    return gr.update(choices=channels, value=channels[:2] if channels else None)  # Pre-select 2 channels

# ✅ Gradio UI Components
with gr.Blocks() as app:
    gr.Markdown("### 🧠 Multi-File EEG Signal Viewer with Unique Colors & ZIP Download")

    # File Upload (Multiple Files)
    file_input = gr.Files(label="Upload EEG CSV Files", file_types=[".csv"])

    # Dynamic Multi-Channel Selection
    channel_selector = gr.CheckboxGroup(label="Select EEG Channels", choices=[], interactive=True)

    # Chunk Size Input
    chunk_input = gr.Number(label="Select Chunk Size", value=256, interactive=True)

    # Process Button
    process_button = gr.Button("Process EEG Data")

    # Output Images
    image_gallery = gr.Gallery(label="Generated EEG Images", columns=3, height=400)

    # Download Button (Initially hidden)
    zip_download = gr.File(label="Download Processed Images", visible=False)

    # Update channel list when files are uploaded
    file_input.change(fn=get_channels, inputs=[file_input], outputs=[channel_selector])

    # Process data when button is clicked
    process_button.click(fn=process_eeg, inputs=[file_input, channel_selector, chunk_input], outputs=[zip_download, image_gallery])

# Run the Gradio app
app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef2f23041eb61552b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import zipfile
from PIL import Image

# 📌 Create base output directory
base_output_dir = "output"
os.makedirs(base_output_dir, exist_ok=True)

# 🎨 **Fixed Color Mapping for Each Channel**
channel_colors = {
    "Delta_TP9": "red", "Delta_AF7": "blue", "Delta_AF8": "green", "Delta_TP10": "purple",
    "Theta_TP9": "orange", "Theta_AF7": "cyan", "Theta_AF8": "magenta", "Theta_TP10": "black",
    "Alpha_TP9": "lime", "Alpha_AF7": "brown", "Alpha_AF8": "pink", "Alpha_TP10": "navy",
    "Beta_TP9": "gold", "Beta_AF7": "darkred", "Beta_AF8": "teal", "Beta_TP10": "indigo",
    "Gamma_TP9": "darkblue", "Gamma_AF7": "darkgreen", "Gamma_AF8": "salmon", "Gamma_TP10": "steelblue",
    "RAW_TP9": "olive", "RAW_AF7": "orchid", "RAW_AF8": "coral", "RAW_TP10": "dodgerblue",
    "AUX_RIGHT": "slategray", "Accelerometer_X": "yellow", "Accelerometer_Y": "chocolate", "Accelerometer_Z": "deepskyblue",
    "Gyro_X": "khaki", "Gyro_Y": "fuchsia", "Gyro_Z": "plum", "HeadBandOn": "mediumspringgreen",
    "HSI_TP9": "chartreuse", "HSI_AF7": "mediumvioletred", "HSI_AF8": "mediumblue", "HSI_TP10": "goldenrod",
    "Battery": "lawngreen", "Elements": "firebrick"
}

# ✅ Function to Process Multiple EEG Files
def process_eeg(files, channels, chunk_size):
    if not files:
        return "❌ Error: Please upload at least one CSV file."

    if not channels:
        return "❌ Error: Please select at least one EEG channel."

    output_folders = []

    # 📌 Process Each Uploaded File
    all_image_files = []
    for file in files:
        df = pd.read_csv(file.name)

        # Ensure valid file
        if df.shape[1] < 2:
            continue

        # Extract Time Column
        time_column = df.columns[0]

        # Extract file name without extension for output folder
        file_name = os.path.basename(file.name).replace('.csv', '')

        # Create a separate folder for each file in the output directory
        output_folder = os.path.join(base_output_dir, file_name)
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)  # Clear old data
        os.makedirs(output_folder)

        output_folders.append(output_folder)

        # 📌 Generate EEG Images for Selected Channels
        total_chunks = len(df) // int(chunk_size)
        image_files = []

        for i in range(total_chunks):
            chunk = df.iloc[i * int(chunk_size) : (i + 1) * int(chunk_size)]

            plt.figure(figsize=(10, 5))

            for channel in channels:
                if channel not in df.columns:
                    continue

                # Plot all selected channels in one graph
                color = channel_colors.get(channel, "black")  # Use mapped color, default to black
                plt.plot(chunk[time_column], chunk[channel], color=color, linestyle='-', linewidth=1, label=channel)

            plt.axis('off')  # Hide X and Y axes
            plt.xticks([])
            plt.yticks([])
            plt.legend(loc="upper right", fontsize=8)  # Add legend to distinguish channels

            # Save Image
            img_filename = f"{output_folder}/EEG_chunk_{i+1}.png"
            plt.savefig(img_filename, bbox_inches='tight', pad_inches=0)
            plt.close()

            image_files.append(img_filename)

        all_image_files.extend(image_files)

    # ✅ Create a ZIP file containing all processed images
    zip_filename = "eeg_processed_data.zip"
    zip_path = os.path.join(base_output_dir, zip_filename)

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for folder in output_folders:
            for root, _, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=os.path.relpath(file_path, base_output_dir))

    return zip_path, all_image_files  # Returns ZIP file & image list

# ✅ Function to Load EEG Channels for Selection (Multi-Select)
def get_channels(files):
    if not files:
        return gr.update(choices=[], value=[])

    df = pd.read_csv(files[0].name)  # Read first file to get channels
    channels = list(df.columns[1:])  # Return all EEG channels (excluding time column)

    return gr.update(choices=channels, value=channels[:2] if channels else None)  # Pre-select 2 channels

# ✅ Gradio UI Components
with gr.Blocks() as app:
    gr.Markdown("### 🧠 Multi-File EEG Signal Viewer with Multi-Channel Plotting & ZIP Download")

    # File Upload (Multiple Files)
    file_input = gr.Files(label="Upload EEG CSV Files", file_types=[".csv"])

    # Dynamic Multi-Channel Selection
    channel_selector = gr.CheckboxGroup(label="Select EEG Channels", choices=[], interactive=True)

    # Chunk Size Input
    chunk_input = gr.Number(label="Select Chunk Size", value=256, interactive=True)

    # Process Button
    process_button = gr.Button("Process EEG Data")

    # Output Images
    image_gallery = gr.Gallery(label="Generated EEG Images", columns=3, height=400)

    # Download Button (Initially hidden)
    zip_download = gr.File(label="Download Processed Images", visible=False)

    # Update channel list when files are uploaded
    file_input.change(fn=get_channels, inputs=[file_input], outputs=[channel_selector])

    # Process data when button is clicked
    process_button.click(fn=process_eeg, inputs=[file_input, channel_selector, chunk_input], outputs=[zip_download, image_gallery])

# Run the Gradio app
app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b547457e015109f9ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
